In [ ]:
''''=============================================================================
     _____ _   _ ______ ______________  ___  ___ _____ _____ _____   ___
    |_   _| \ | ||  ___|  _  | ___ \  \/  | / _ \_   _|_   _/  __ \ / _ \
      | | |  \| || |_  | | | | |_/ / .  . |/ /_\ \| |   | | | /  \// /_\ \
      | | | . ` ||  _| | | | |    /| |\/| ||  _  || |   | | | |    |  _  |
     _| |_| |\  || |   \ \_/ / |\ \| |  | || | | || |  _| |_| \__/\| | | |
     \___/\_| \_/\_|    \___/\_| \_\_|  |_/\_| |_/\_/  \___/ \____/\_| |_/
                             _   _ _   _ _________________
                            | | | | \ | |_   _| ___ \ ___ \
                            | | | |  \| | | | | |_/ / |_/ /
                            | | | | . ` | | | |  __/|    /
                            | |_| | |\  |_| |_| |   | |\ \
                             \___/\_| \_/\___/\_|   \_| \_|
===============================================================================
===============================================================================
PROJECT:                  -Solve sudoku 2x2 with Grover in quantum computing-
LANGUAGE:                 python notebook
SOURCE NAME:              sudoku_2x2_solver.ipynb
VERSION:                  1.0
DATE:                     14/01/2025
===============================================================================
===================================AUTHORS=====================================
===============================================================================
ARIANNA CIPOLLA     379818      arianna.cipolla@studenti.unipr.com
AURORA FELISARI     379867      aurora.felisari@studenti.unipr.com
CLAUDIO BENDINI     380972      claudio.bendini@studenti.unipr.com
===============================================================================
==============================PACKAGES VERSION=================================
===============================================================================
    jupyter                   1.1.1
    jupyterlab                4.3.4
    matplotlib                3.10.0
    matplotlib-inline         0.1.7
    numpy                     2.2.1
    qiskit                    1.3.1
    qiskit-aer                0.15.1
    rustworkx                 0.15.1
    scipy                     1.15.1
===============================================================================
=================================CHANGELOGS====================================
===============================================================================
1.0
    - First release
    Known bug:
        A. Enablin both depolarization and relaxation noise will not add both.
        B. Histogram shows randomly ignoring clause.
        C. Wrapping qc.draw(output="mpl") in an if stop displaying entirely.
    TODO:
        A. Implement noise prints.
============================================================================='''
#######################################
#               MODULES
#######################################

from datetime import datetime
from operator import itemgetter
from collections import Counter
import numpy as np
import math
import os
import sys

from qiskit import *
from qiskit import qpy
from qiskit_aer import QasmSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error, thermal_relaxation_error, ReadoutError
from qiskit.visualization import plot_histogram
#######################################
#           USER DASHBOARD
#######################################

# Flags for enabling specific errors noises
depolarization_noise_on = True     # Toggle depolarization noise
relaxation_noise_on = False        # Toggle relaxation noise
measurement_error_on = True        # Toggle measurement error

# Flags for activating mitigations
reduce_random = False               # Run the circuit multiple times to reduce random errors

# Flags for enabling output files
save_circuit = True            # Save the quantum circuit to file
save_histogram = True          # Save the histogram of results to file
save_solutions = True          # Save the solutions to file
save_noise = True              # TODO: Save noise model details to file

# Flags for displaying output
show_solutions = True          # Print solutions to the console
show_noise = True              # TODO: Print noise model details to the console
show_histogram = True          # Display the histogram of results
show_circuit = True            # Display circuit picture

# Simulation configurations
reversibility_disable = False   # !WARNING! Tests the essentiality of reversibility
simulation_shots = 10024        # Number of shots for the simulation // higher number reduces statistical noise
num_repeats = 100                # Random reduction mitigation repeats
grover_iterations = 2           # Set 0 to use the suggested optimal value

#######################################
#               FUNCTIONS
#######################################

def suggested_iterations(number_of_iterations):
    """Calculate the suggested number of Grover iterations."""
    row_columns = 2

    solutions_space = math.pow(row_columns,math.pow(row_columns,2))
    acceptable_solutions = math.factorial(row_columns)*math.factorial(row_columns - 1)

    number_of_iterations = math.pi * math.sqrt(solutions_space/acceptable_solutions) / 4
    number_of_iterations = math.ceil(number_of_iterations)
    print(f"Grover's itertions: {number_of_iterations}.\n")
    return number_of_iterations

# Apply superposition to input qubits
def apply_superposition(qubits):
    """Apply Hadamard gates to create superposition."""
    for qubit in qubits:
        qc.h(qubit[0])

# Flip the state of the qubits
def flip_qubits(qubits):
        """Apply X gates to flip qubit states."""
        for qubit in qubits:
            qc.x(qubit[0])

# Compare two qubits using an ancilla qubit
def compare(qc, control_qubit, target_qubit, ancilla_qubit):
    """Perform controlled comparison between two qubits."""
    qc.x(target_qubit)
    qc.mcx([control_qubit, target_qubit], ancilla_qubit)
    qc.x(target_qubit)

# Perfomr a XOR operation to two given qubits and returns result in ancilla
def XOR(qc, qubit_1, qubit_2, ancilla):
    """Perform XOR operation using ancilla qubits."""
    if(reversibility_disable):
        qc.cx(qubit_1[0], qubit_2[0])
        qc.barrier()
        return

    compare(qc, qubit_1[0], qubit_2[0], ancilla[0])
    compare(qc, qubit_2[0], qubit_1[0], ancilla[1])
    qc.cx(ancilla[0], ancilla[1])
    qc.barrier()

# Perform inversion about the mean
def inversion_about_mean(qubits, target_qubit):
    """Perform inversion about the mean operation."""
    qc.h(target_qubit[0])
    qc.mcx([qubit for qubit in qubits if qubit != target_qubit], target_qubit[0])
    qc.h(target_qubit[0])

# Define the oracle for Grover's algorithm
def oracle(qc):
    """Mark the desired states using XOR gates."""
    XOR(qc, qubits[0], qubits[1], ancilla_ab)
    XOR(qc, qubits[0], qubits[2], ancilla_ac)    
    XOR(qc, qubits[1], qubits[3], ancilla_bd)
    XOR(qc, qubits[2], qubits[3], ancilla_cd)

# Diffuser operation for Grover's algorithm
def diffuser(qc):
    """Apply the diffuser transformation for Grover's algorithm."""
    apply_superposition(qubits)
    flip_qubits(qubits)

    inversion_about_mean(qubits, qubits[0])

    flip_qubits(qubits)
    apply_superposition(qubits)

    qc.barrier()

# Print Sudoku solutions to console and file
def solution_toString(dict):
    """Print solutions in a readable format."""
    list = []
    for index, item in enumerate(dict, start=1):
        for a in item:
            list.append(a)
        data = np.array(list)
        shape = (2, 2)
        sudoku = data.reshape(shape)
        if save_solutions:
            original_stdout = sys.stdout
            print(f"Solution {index}:\n {sudoku}\n")
            sys.stdout = open(f"{newpath}/solutions_{timestamp}.txt", 'a')
            print(f"Solution {index}:\n {sudoku}\n")
            sys.stdout = original_stdout
        list.clear()
        
#######################################
#               MAIN
#######################################

if grover_iterations == 0:
    grover_iterations = suggested_iterations(grover_iterations)

# Define qubits and ancilla registers
qubits = []
for index in range(4):
    qubits.append(QuantumRegister(1, name=f"cell_{bin(index + 4)[3:5]}"))

ancilla_ab = QuantumRegister(2, name="ancilla ab")
ancilla_cd = QuantumRegister(2, name="ancilla cd")
ancilla_ac = QuantumRegister(2, name="ancilla ac")
ancilla_bd = QuantumRegister(2, name="ancilla bd")

quantum_output = QuantumRegister(1,name="output")
classic_output = ClassicalRegister(4, name="measure")

# Create quantum circuit
qc = QuantumCircuit(qubits[0], qubits[1], qubits[2], qubits[3], ancilla_ab,
                    ancilla_cd, ancilla_ac, ancilla_bd, quantum_output, classic_output)

# Initialize quantum circuit
qc.x(quantum_output)  # Initialize output to |1>
qc.h(quantum_output)  # Apply Hadamard to output

apply_superposition(qubits)

qc.barrier()

# Grover's algorithm loop
for i in range(grover_iterations):
    oracle(qc)
    qc.mcx([ancilla_ab[1], ancilla_ac[1], ancilla_cd[1], ancilla_bd[1]], quantum_output) #toffoli?
    qc.barrier()
    oracle(qc)
    diffuser(qc)

# Measure output
for index, qubit in enumerate(qubits, start=0):
    qc.measure(qubit[0], classic_output[index])

# Create noise model
custom_noise_model = NoiseModel()

# --- noise: DEPOLARIZATION ---
if depolarization_noise_on:
    one_qubit_error = depolarizing_error(param=0.02, num_qubits=1)  # 1% error for 1-qubit gates
    two_qubit_error = depolarizing_error(param=0.03, num_qubits=2)  # 2% error for 2-qubit gates (cx)

    custom_noise_model.add_all_qubit_quantum_error(one_qubit_error, ['x', 'h', 'u3'])  # 1-qubit gates
    custom_noise_model.add_all_qubit_quantum_error(two_qubit_error, ['cx'])  # 2-qubit gates

# --- noise: RELAXATION ---
if relaxation_noise_on:
    # Apply relaxation error only to 1-qubit gates (not multi-qubit gates)
    T1s = np.random.normal(50e3, 10e3, 4) # Sampled from normal distribution mean 50 microsec
    T2s = np.random.normal(70e3, 10e3, 4)  # Sampled from normal distribution mean 50 microsec

    # Truncate random T2s <= T1s
    T2s = np.array([min(T2s[j], 2 * T1s[j]) for j in range(4)])

    # Instruction times (in nanoseconds)
    time_u1 = 0   # virtual gate
    time_u2 = 50  # (single X90 pulse)
    time_u3 = 100 # (two X90 pulses)
    time_cx = 300
    time_reset = 1000  # 1 microsecond
    time_measure = 1000 # 1 microsecond

    # QuantumError objects
    errors_reset = [thermal_relaxation_error(t1, t2, time_reset) for t1, t2 in zip(T1s, T2s)]
    errors_measure = [thermal_relaxation_error(t1, t2, time_measure) for t1, t2 in zip(T1s, T2s)]
    errors_u1  = [thermal_relaxation_error(t1, t2, time_u1) for t1, t2 in zip(T1s, T2s)]
    errors_u2  = [thermal_relaxation_error(t1, t2, time_u2) for t1, t2 in zip(T1s, T2s)]
    errors_u3  = [thermal_relaxation_error(t1, t2, time_u3) for t1, t2 in zip(T1s, T2s)]
    errors_cx = [[thermal_relaxation_error(t1a, t2a, time_cx).expand(
                thermal_relaxation_error(t1b, t2b, time_cx))
                for t1a, t2a in zip(T1s, T2s)]
                for t1b, t2b in zip(T1s, T2s)]

    for j in range(4):
        custom_noise_model.add_quantum_error(errors_reset[j], "reset", [j])
        custom_noise_model.add_quantum_error(errors_measure[j], "measure", [j])
        custom_noise_model.add_quantum_error(errors_u1[j], "u1", [j])
        custom_noise_model.add_quantum_error(errors_u2[j], "u2", [j])
        custom_noise_model.add_quantum_error(errors_u3[j], "u3", [j])
        for k in range(4):
            custom_noise_model.add_quantum_error(errors_cx[j][k], "cx", [j, k])


# --- noise: READOUT ---
if measurement_error_on:
    readout_error_a = ReadoutError([[0.90, 0.10], [0.05, 0.95]])  # Readout error for qubit a
    readout_error_b = ReadoutError([[0.90, 0.10], [0.05, 0.95]])  # Readout error for qubit b
    readout_error_c = ReadoutError([[0.90, 0.10], [0.05, 0.95]])  # Readout error for qubit c
    readout_error_d = ReadoutError([[0.90, 0.10], [0.05, 0.95]])  # Readout error for qubit d

    # Apply readout errors to measurements for each qubit
    custom_noise_model.add_readout_error(readout_error_a, [0])  # Apply to qubit a_string[0]
    custom_noise_model.add_readout_error(readout_error_b, [1])  # Apply to qubit b_string[0]
    custom_noise_model.add_readout_error(readout_error_c, [2])  # Apply to qubit c_string[0]
    custom_noise_model.add_readout_error(readout_error_d, [3])  # Apply to qubit d_string[0]

# --- Perform simulation ---
# Start simulation
backend = AerSimulator()

# Transpile the circuit for the selected backend
transpiled_circuit = transpile(qc, backend)

if reduce_random:
    # Run the transpiled circuit with noise model
    all_counts = []
    for _ in range(num_repeats):
        result = backend.run(transpiled_circuit, shots=simulation_shots, noise_model=custom_noise_model).result()
        all_counts.append(result.get_counts())

    average_counts = Counter()
    for counts in all_counts:
        average_counts.update(counts)

    # Normalize the counts
    total_shots = sum(average_counts.values())
    raw_data = {key: value / total_shots for key, value in average_counts.items()}
else:
    result = backend.run(transpiled_circuit, shots=simulation_shots, noise_model=custom_noise_model).result()

# Get the result counts
raw_data = result.get_counts()

# --- Post process gathered data ---
# Generate timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")  # Format: YYYYMMDD_HHMMSS

# Generate folder for report
conditions = f"{simulation_shots}_"
if depolarization_noise_on:
    conditions += "D"
if relaxation_noise_on:
    conditions += "R"
if measurement_error_on:
    conditions += "M"

newpath = fr"reports/Sudoku_2x2/{conditions}"
if not os.path.exists(newpath):
    os.makedirs(newpath)

# Plot histogram
fig = plot_histogram(raw_data)

if save_histogram:
    fig.figure.savefig(f"{newpath}/histogram_{timestamp}.png", dpi=300)
''' if show_histogram:
        fig.show() # BUG:DOESN'T WORK'''

# Get the top N results
results = dict(sorted(raw_data.items(), key=itemgetter(1), reverse=True)[:2])

# Save and shows solutions
if save_solutions or show_solutions:
    with open(f"{newpath}/solutions_{timestamp}.txt", "a") as file:
        if show_solutions:
            print(solution_toString(results), file)
        else:
            solution_toString(results)

# Save circuit.qpy to file
if save_circuit:
    with open(f"{newpath}/circuit_{timestamp}.qpy", "wb") as file:
        qpy.dump(qc, file)

# Save circuit picture to file
if save_circuit:
    qc.draw(output="mpl", filename=f"{newpath}/circuit_{timestamp}.png")

''' if show_circuit:
        qc.draw(output="mpl") # BUG:DOESN'T WORK'''

qc.draw(output="mpl")